In [2]:
%run 0.0_Dependance.ipynb

In [3]:
df = pd.read_csv("train.csv",index_col=0)
df["N_Year"]=np.round((df['N_Days']/365),2)
df.Age = np.round((df.Age)/365,1)

#label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
colonne_a_encoder = ["Sex","Ascites","Hepatomegaly","Spiders","Edema","Drug","Status"]	
for i in colonne_a_encoder:
    df[i] = le.fit_transform(df[i])

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report

target = df.Status
data = df.drop(columns=["Status","N_Days"])

X_train,X_test,y_train,y_test = train_test_split(data,target,test_size=0.2, random_state=123)
#Standardisation
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [4]:
df_score_random = load("df_score_random.joblib")
df_score_random

,n_estimators,max_depth,learning_rate,gamma,subsample,colsample_bytree,min_child_weight,reg_alpha,reg_lambda,objective,eval_metric,booster,LogLoss_mean,LogLoss_std
0,250,9,0.01,0.1,0.75,1,10,1,1,multi:softprob,mlogloss,gbtree,-0.495877,0.007797
1,100,3,0.2,0.5,1,1,1,0.1,10,multi:softprob,mlogloss,gbtree,-0.448388,0.014140
2,1000,9,0.05,1,0.5,0.5,10,1,10,multi:softprob,mlogloss,gbtree,-0.440227,0.015089
3,500,12,0.2,0,0.75,0.75,5,0,0,multi:softprob,mlogloss,gbtree,-0.619045,0.030493
4,500,9,0.2,1,0.5,1,5,0,10,multi:softprob,mlogloss,gbtree,-0.466344,0.015956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30495,50,6,0.05,0,0.5,0.75,5,1,0,multi:softprob,mlogloss,gbtree,-0.496917,0.006187
30496,1000,3,0.1,1,0.75,1,1,0.1,10,multi:softprob,mlogloss,gbtree,-0.446553,0.013987
30497,500,9,0.1,0,0.5,0.75,10,0,0,multi:softprob,mlogloss,gbtree,-0.506701,0.021323
30498,100,9,0.2,0.1,1,1,5,0.1,1,multi:softprob,mlogloss,gbtree,-0.477485,0.015654


In [ ]:
df_score_random = df_score_random.sort_values(by="LogLoss_mean",ascending=False)
df_score_random_top_1000 = df_score_random.iloc[:1000,:]

list_log_loss=[]
for i in range(1000):
    df = df_score_random_top_1000.iloc[i,:-2]
    dico = df.to_dict()
    model = XGBClassifier(**dico)
    model.fit(X_train, y_train)
    probs = model.predict_proba(X_test)
    logloss = log_loss(y_test, probs)
    list_log_loss.append(logloss)
    
df_score_random_top_1000['Test_log_loss']=list_log_loss